<div align="center">

---
---
# 📚 Proyecto Final Programa Experto Data Science 06
---
---
#### Datamecum
#### Autor: Carlos Zamora

</div>

## Objetivo del proyecto

🎯 La naturaleza de la problemática presentada es de Regresión (Aprendizaje Supervisado). El objetivo principal es la obtención de un modelo que se ajuste a los datos proporcionados, y tenga buena capacidad de predicción.

---
## 📊 Fase 1: EDA

💡Dado que los datos están ofuscados, se descarta aplicar conocimiento de dominio a los mismos.

🔍 Comenzamos calculando los estadísticos básicos del dataset y graficando las entidades importantes (relaciones entre variables, distribuciones, etc.)

IMPORTANTE: debido al uso de Widgets, este notebook se mostrará sin outputs en GitHub, ya que no se pueden renderizar. Para ver el análisis completo, se recomienda descargar el proyecto y ejecutarlo en local.

---

In [ ]:
import pandas as pd
import numpy as np
import seaborn as sns
from ipywidgets import interact, fixed
import missingno as msno

import sys
sys.path.append('..\src')
from functions import (unique_col_values, 
                           test_anderson_norm, 
                           heatmap_corr,
                           reset_test,
                           grafico_caja
                           )

In [ ]:
datos = pd.read_excel('../data/train.xlsx')
datos.head()

In [ ]:
datos.info()
datos.describe()

In [ ]:
unique_col_values(datos)

In [ ]:
datos['x8'].value_counts().sort_index()

---
Con este primer vistazo a los datos, podemos aseverar varios puntos:
1. Hay una **variable extremadamente sesgada ("x1")**, con una diferencia muy grande en valor entre media y mediana; esto representa potencial incidencia de no normalidad. Además, tiene una distribución muy amplia (mucha distancia entre cuantiles); esto refleja la posible existencia de valores atípicos.
2. Dos variables **("x2" y "x3")** presentan **indicios de una distribución bimodal**, por la distribución de sus cuartiles. Podría indicar que datos proceden de 2 distribuciones distintas para cada una.
3. **"x10"** parece ser una variable de tipo porcentaje, como un **coeficiente o un ratio**.
4. **"x8"** presenta solamente **15 valores distintos**. Además, estos 15 valores parecen ser días del año, ya que el rango va desde 1 a 365. Podría ser un indicador de la sucesión de un evento.
5. Presencia de **nulos en todas las variables excepto en la "deseada"**.
---

In [ ]:
msno.heatmap(datos, figsize=(10,5), fontsize=10)

In [ ]:
msno.matrix(datos)

---
El análisis de nulos en detalle revela que el proceso que explica los datos faltantes podría ser **MCAR o MAR**. El MCAR es más una idealización teórica que un factor explicativo de datos faltantes en un contexto real, por tanto asumimos que existe un proceso [(Missing at Random)](https://stefvanbuuren.name/fimd/sec-MCAR.html) que los explica, y **sería correcto aplicar técnicas de imputación**.

---

In [ ]:
bplot = sns.pairplot(datos, height=2.5, diag_kind='kde')
bplot.fig.suptitle("Relaciones entre variables y distribución (para agrandar,\
hacer clic en la imagen)", fontsize=16, fontweight='bold', y=1.02)

---
De la parrilla anterior pueden extraerse varios **insights adicionales**:
1. La **distribución** de las variables es, en general, no normal, a excepción de algunas ("x9").
2. La **dispersión** solo muestra una relación más o menos clara entre las variables independientes "x4" y "x5". Además, dicha relación parece ser de carácter polinómico. El resto de nubes de puntos no siguen ningún patrón estilizado.
3. En cuanto a las **distribuciones bimodales** que parecían intuirse mediante estadísticos relevantes, la única que se sostiene parece ser la variable "x3", que tiene una dualidad muy marcada.

---

In [ ]:
dict_anderson = test_anderson_norm(datos)
df_anderson = pd.DataFrame.from_dict(dict_anderson, orient='index')
df_anderson.head(10)

---
Si comparamos el estadístico que devuelve el test con los valores críticos para los distintos niveles de significatividad, comprobamos que, **exceptuando la variable "x9", se rechaza la hipótesis nula de normalidad** a todos los niveles de significancia.

---

In [ ]:
interact(
   heatmap_corr,
   df=fixed(datos),
   metodologia=['spearman', 'pearson']
)

---
Utilizando el método de rangos de Spearman, robusto a no normalidad, vemos que existe **correlación fuerte entre "x4" y "x5", y entre "x8" y la "deseada"**.

La existencia de alta correlación entre la variable regresada y una regresora no es problemático de cara a realizar predicciones. Sin embargo, si en la fase del modelado optásemos por uno de regresión lineal, habrá que considerar la posible multicolinealidad derivada de la correlación entre "x4" y "x5".

---

In [ ]:
# Dado que en nuestros datos existen indicios de relaciones no lineales entre
# variables, vamos a aplicar el test RESET de Ramsey para corroborarlo
y = "deseada"
X = list(datos.drop(columns="deseada").columns)

In [ ]:
interact(
   reset_test,
   df=fixed(datos),
   X=fixed(X),
   y=fixed(y),
   potencia=range(2,10)
)

---
Se ha detectado la **presencia de relaciones** polinómicas de **hasta cuarto orden**

---

In [ ]:
interact(
   grafico_caja,
   df=fixed(datos),
   column=datos.columns.tolist()
)

---
A **excepción de "x3", "x6" y "x10", todas las variables presentan valores atípicos**.

---

---
## 📝 Conclusiones principales EDA

- Todas las variables excepto la regresada presentan **nulos, siguiendo patrón MAR** --> se puede aplicar imputación para su subsanación
- Alta presencia de **no normalidad** --> problemas en modelos de reg. lineal por violación de supuestos Gauss-Markov si la no-normalidad se traslada a los errores
- Variable **x8** potencialmente categórica (15 valores únicos sobre 772 registros). Variable **x1** candidata a aplicarle *binnization* debido a la amplitud de su distribución y los saltos discretos que se aprecian entre quintiles.
- **Correlaciones** en el umbral problemático para las variables "x4" y "x5" --> a vigilar si se elige un modelo de reg. lineal por posible multicolinealidad
- Presencia de **relaciones polinómicas** entre las X y la y (hasta de grado 4) --> a tener en cuenta si se elige un modelo lineal para predecir
- **Outliers** en casi todos los atributos --> problemas en modelos de reg. lineal por inflación de varianza, que hace que las β dejen de ser MELI. En otro tipo de modelos, como los basados en árboles, no es tan problemático, pero aún así se aplicará escalado robusto en la fase posterior.
---

---
Para finalizar con la fase de EDA, se va a proceder a agrupar la variable 'x1' en *bins*, debido a los hallazgos durante la fase exploratoria.

Se va a optar por agrupar la variable en cuartiles, buscando un compromiso entre generalidad y granularidad, y sobre todo poniendo énfasis en no sobreajustar la distribución más adelante en la fase de modelizado.

---

In [ ]:
datos['x1_bin'] = pd.qcut(x=datos['x1'], q=4, labels=False, duplicates='drop')
df_pipeline = datos.drop(columns='x1')
df_pipeline.to_csv('df_pipeline.csv')